In [ ]:
# Dependencies and Setup
import pandas as pd
import requests
# import numpy as np
import io
import ipyleaflet
import json
# from pathlib import Path
# from pprint import pprint
from ipyleaflet import Map, Marker, LayersControl, SearchControl, FullScreenControl, WidgetControl, GeoJSON, LayerGroup, Popup
# from geopy.exc import GeocoderTimedOut 
# from geopy.geocoders import Nominatim 
# from ipywidgets import link, FloatSlider
from branca.colormap import linear
from pandas_geojson import to_geojson
from ipywidgets import HTML, widgets
from shapely.geometry import shape

In [ ]:
# Downloading the csv file from the GitHub repository
url = "https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/coords.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
coords_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

# Printing out the first 5 rows of the dataframe
coords_df.head()

In [ ]:
# Downloading the csv file from the GitHub repository
url = "https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/2015.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
data_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

# Printing out the first 5 rows of the dataframe
data_df.head()

In [ ]:
# Join the dataframes
data_df = pd.merge(happy15_df, coords_df, on='Country', how='inner')
data_df.head()

In [ ]:
def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)

In [ ]:
data = load_data(
    'https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/2015.geo.json',
    '2015.geo.json',
     json.load)

In [ ]:
# Load the choropleth map
layer = ipyleaflet.Choropleth(
    geo_data= data,
    choro_data=dict(zip(data_df['Country'].str[:2].str.upper().tolist(), data_df['Happiness Score'].tolist())),
    colormap=linear.YlOrRd_04,
    border_color='white',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'},
    name= '2015'
)
m2 = ipyleaflet.Map(center = (0,0), zoom = 1)
m2.add(layer)

In [ ]:
control = LayersControl(position='topright')
m.add(control)

In [ ]:
m.add(FullScreenControl())

In [ ]:
# m.save('C:/Users/Brown/Documents/my_map.html', title='My Map')

In [ ]:
# m.clear()

In [ ]:
# Search in a GeoJSON 

countries = GeoJSON(data=data)

# m = Map(center=[61.9389, 69.7851], zoom=2)

search = SearchControl(
    position="topleft",
    layer=LayerGroup(layers=(countries,)),
    property_name="Country",
)
m.add(search)
m

In [ ]:
def add_popup(feature, **kwargs):
    for layer in m.layers:
        if isinstance(layer, Popup):
            m.remove_layer(layer)
    p = shape(feature["geometry"])
    center = p.centroid.y, p.centroid.x
    properties = feature["properties"]
    message = HTML()
    message.value = f"""<h4>{properties["Country"]}</h4>
                        <b>Happiness Score: {properties["Happiness Score"]}
                     """
    popup = Popup(
        location=center,
        child=message,
        close_button=True,
        auto_close=False,
        close_on_escape_key=False,
    )

    m.add(popup)


search.on_feature_found(add_popup)

In [ ]:
# TESTING
w = widgets.Dropdown(
    options=['m', 'm2'],
    value = m,
    description='List',
)
display(w)

# def on_change(change):
#     if change['name'] == 'value' and (change['new'] != change['old']):
#         clear_output()
#         m

# w.observe(on_change)